# Base de Datos SQL Relacional con Python

Borja Barber Lead Instructor DS


## Introducción

En este tutorial aprenderemos a crear y gestionar bases de datos SQL relacionales usando Python.

Usaremos **SQLite**, una base de datos ligera que viene incluida con Python y no requiere instalación adicional.

### Ejemplo: Sistema de Tienda Online

Crearemos una base de datos para una tienda online con las siguientes tablas:
- **Clientes**: información de los clientes
- **Productos**: catálogo de productos
- **Pedidos**: pedidos realizados
- **Detalles_Pedido**: productos incluidos en cada pedido

## Paso 1: Importar las librerías necesarias

In [ ]:
# Importamos sqlite3, la librería estándar de Python para trabajar con bases de datos SQLite
# SQLite es una base de datos embebida que no requiere servidor
import sqlite3

# Importamos datetime para trabajar con fechas y horas
# Esto nos permitirá registrar cuándo ocurren eventos (ej: fecha de registro de cliente)
from datetime import datetime

# Importamos pandas para visualizar mejor los resultados (opcional pero recomendado)
# Pandas nos permite mostrar los datos en formato tabla más legible
import pandas as pd

# Mensaje de confirmación para saber que todo se importó correctamente
print("Librerías importadas correctamente")

## Paso 2: Crear la conexión a la base de datos

Para trabajar con SQLite necesitamos:
1. **Conexión**: objeto que representa la conexión con la base de datos
2. **Cursor**: objeto que nos permite ejecutar comandos SQL

In [ ]:
# Creamos la conexión a la base de datos
# sqlite3.connect() crea un archivo .db si no existe o se conecta a uno existente
# Si el archivo 'tienda_online.db' no existe, se creará automáticamente
conexion = sqlite3.connect('tienda_online.db')

# Creamos un cursor, que es el objeto que usaremos para ejecutar comandos SQL
# El cursor es como un "puntero" que nos permite navegar y modificar la base de datos
cursor = conexion.cursor()

# Mensajes informativos para confirmar que la conexión fue exitosa
print("Conexión establecida exitosamente")
print(f"Base de datos: tienda_online.db")

## Paso 3: Crear la tabla CLIENTES

Comenzamos creando la tabla de clientes con los siguientes campos:
- **id_cliente**: identificador único (clave primaria)
- **nombre**: nombre del cliente
- **email**: correo electrónico (único)
- **telefono**: número de teléfono
- **direccion**: dirección de envío
- **fecha_registro**: fecha en que se registró el cliente

In [ ]:
# Primero eliminamos la tabla si ya existe (útil para reiniciar desde cero)
# DROP TABLE IF EXISTS es una operación segura que no da error si la tabla no existe
cursor.execute('DROP TABLE IF EXISTS Clientes')

# Creamos la tabla Clientes con SQL usando cursor.execute()
# Tipos de datos en SQLite:
#   - INTEGER: números enteros
#   - TEXT: cadenas de texto
#   - REAL: números decimales
#   - DATE: fechas
# Restricciones:
#   - PRIMARY KEY: identifica de forma única cada registro
#   - AUTOINCREMENT: genera automáticamente el ID (1, 2, 3, ...)
#   - NOT NULL: el campo no puede estar vacío (es obligatorio)
#   - UNIQUE: no puede haber valores duplicados (ej: dos emails iguales)
cursor.execute('''
    CREATE TABLE Clientes (
        id_cliente INTEGER PRIMARY KEY AUTOINCREMENT,
        nombre TEXT NOT NULL,
        email TEXT NOT NULL UNIQUE,
        telefono TEXT,
        direccion TEXT,
        fecha_registro DATE NOT NULL
    )
''')

# Guardamos los cambios en la base de datos
# commit() hace permanentes los cambios - sin esto, los cambios se pierden
conexion.commit()

print("✓ Tabla CLIENTES creada exitosamente")

## Paso 4: Crear la tabla PRODUCTOS

Ahora creamos la tabla de productos para nuestro catálogo.

In [ ]:
# Eliminamos la tabla Productos si existe (para empezar limpio)
cursor.execute('DROP TABLE IF EXISTS Productos')

# Creamos la tabla Productos
# REAL: tipo de dato para números decimales (ideal para precios como 19.99)
# INTEGER: tipo de dato para números enteros (ideal para cantidad de stock)
# DEFAULT 0: si no especificamos el stock, por defecto será 0
cursor.execute('''
    CREATE TABLE Productos (
        id_producto INTEGER PRIMARY KEY AUTOINCREMENT,
        nombre TEXT NOT NULL,
        descripcion TEXT,
        precio REAL NOT NULL,
        stock INTEGER NOT NULL DEFAULT 0,
        categoria TEXT
    )
''')

# Guardamos los cambios con commit()
conexion.commit()

print("✓ Tabla PRODUCTOS creada exitosamente")

## Paso 5: Crear la tabla PEDIDOS

Esta tabla almacenará la información general de cada pedido.

Incluye una **clave foránea** (FOREIGN KEY) que relaciona cada pedido con un cliente.

In [ ]:
# Eliminamos la tabla Pedidos si existe
cursor.execute('DROP TABLE IF EXISTS Pedidos')

# Creamos la tabla Pedidos
# Esta tabla contiene información general de cada pedido
# FOREIGN KEY: establece una relación con la tabla Clientes
#   - id_cliente debe existir en la tabla Clientes (integridad referencial)
# ON DELETE CASCADE: si eliminamos un cliente, automáticamente se eliminan sus pedidos
#   - Esto mantiene la consistencia de los datos
cursor.execute('''
    CREATE TABLE Pedidos (
        id_pedido INTEGER PRIMARY KEY AUTOINCREMENT,
        id_cliente INTEGER NOT NULL,
        fecha_pedido DATE NOT NULL,
        estado TEXT NOT NULL DEFAULT 'Pendiente',
        total REAL NOT NULL,
        FOREIGN KEY (id_cliente) REFERENCES Clientes(id_cliente) ON DELETE CASCADE
    )
''')

# Guardamos los cambios en disco
conexion.commit()

print("✓ Tabla PEDIDOS creada exitosamente")

## Paso 6: Crear la tabla DETALLES_PEDIDO

Esta tabla relaciona pedidos con productos (relación muchos a muchos).

Cada registro indica qué productos y en qué cantidad están en cada pedido.

In [ ]:
# Eliminamos la tabla Detalles_Pedido si existe
cursor.execute('DROP TABLE IF EXISTS Detalles_Pedido')

# Creamos la tabla Detalles_Pedido
# Esta tabla relaciona pedidos con productos (tabla intermedia)
# Resuelve la relación muchos a muchos:
#   - Un pedido puede tener muchos productos
#   - Un producto puede estar en muchos pedidos
# Esta tabla tiene DOS claves foráneas:
#   - FOREIGN KEY hacia Pedidos: conecta con el pedido al que pertenece
#   - FOREIGN KEY hacia Productos: conecta con el producto vendido
# ON DELETE CASCADE: si eliminamos un pedido o producto, se eliminan sus detalles
cursor.execute('''
    CREATE TABLE Detalles_Pedido (
        id_detalle INTEGER PRIMARY KEY AUTOINCREMENT,
        id_pedido INTEGER NOT NULL,
        id_producto INTEGER NOT NULL,
        cantidad INTEGER NOT NULL,
        precio_unitario REAL NOT NULL,
        subtotal REAL NOT NULL,
        FOREIGN KEY (id_pedido) REFERENCES Pedidos(id_pedido) ON DELETE CASCADE,
        FOREIGN KEY (id_producto) REFERENCES Productos(id_producto) ON DELETE CASCADE
    )
''')

# Guardamos los cambios
conexion.commit()

print("✓ Tabla DETALLES_PEDIDO creada exitosamente")

## Paso 7: Verificar las tablas creadas

Vamos a consultar el esquema de la base de datos para verificar que todas las tablas se crearon correctamente.

In [ ]:
# Consultamos la tabla sqlite_master que contiene información sobre todas las tablas
# sqlite_master es una tabla especial del sistema que almacena metadatos
# WHERE type='table' filtra solo las tablas (no vistas, índices, etc.)
cursor.execute("SELECT name FROM sqlite_master WHERE type='table'")

# fetchall() obtiene todos los resultados de la consulta como una lista de tuplas
tablas = cursor.fetchall()

print("Tablas en la base de datos:")
print("="*40)
# Iteramos sobre cada tabla encontrada
for tabla in tablas:
    # tabla[0] contiene el nombre de la tabla (primera columna del resultado)
    print(f"  • {tabla[0]}")

## Paso 8: Insertar datos en CLIENTES

Ahora vamos a insertar registros en nuestras tablas.

### Método 1: Insertar un solo registro

In [ ]:
# Insertamos un cliente usando el comando INSERT INTO
# INSERT INTO especifica la tabla y las columnas donde insertar datos
# VALUES especifica los valores a insertar
# Los signos de interrogación (?) son marcadores de posición (placeholders)
#   - Evitan inyección SQL (un problema de seguridad grave)
#   - SQLite reemplaza cada ? por el valor correspondiente de la tupla
cursor.execute('''
    INSERT INTO Clientes (nombre, email, telefono, direccion, fecha_registro)
    VALUES (?, ?, ?, ?, ?)
''', ('Juan Pérez', 'juan.perez@email.com', '666123456', 'Calle Mayor 10, Madrid', '2024-01-15'))

# Guardamos los cambios con commit()
# Sin commit(), los cambios solo están en memoria y se perderían
conexion.commit()

print("✓ Cliente insertado correctamente")
# lastrowid nos da el ID autoincremental del último registro insertado
print(f"ID del cliente insertado: {cursor.lastrowid}")

### Método 2: Insertar múltiples registros

In [ ]:
# Creamos una lista de tuplas con los datos de varios clientes
# Cada tupla representa un cliente con sus datos en el mismo orden que las columnas
clientes_datos = [
    ('María García', 'maria.garcia@email.com', '677234567', 'Avenida Libertad 25, Barcelona', '2024-02-20'),
    ('Carlos Rodríguez', 'carlos.rodriguez@email.com', '688345678', 'Plaza España 5, Valencia', '2024-03-10'),
    ('Ana Martínez', 'ana.martinez@email.com', '699456789', 'Calle Sol 15, Sevilla', '2024-03-25'),
    ('Luis Fernández', 'luis.fernandez@email.com', '611567890', 'Calle Luna 8, Bilbao', '2024-04-05')
]

# executemany() ejecuta la misma consulta múltiples veces con diferentes datos
# Es más eficiente que ejecutar execute() en un bucle
# Inserta todos los clientes de la lista en una sola operación
cursor.executemany('''
    INSERT INTO Clientes (nombre, email, telefono, direccion, fecha_registro)
    VALUES (?, ?, ?, ?, ?)
''', clientes_datos)

# Guardamos todos los cambios a la vez
conexion.commit()

print(f"✓ {len(clientes_datos)} clientes insertados correctamente")

## Paso 9: Insertar datos en PRODUCTOS

In [ ]:
# Lista de productos para nuestra tienda
# Cada tupla contiene: (nombre, descripcion, precio, stock, categoria)
# Los datos deben estar en el mismo orden que las columnas del INSERT
productos_datos = [
    ('Laptop HP Pavilion', 'Portátil 15.6 pulgadas, Intel i5, 8GB RAM', 699.99, 15, 'Informática'),
    ('Mouse Logitech MX', 'Mouse inalámbrico ergonómico', 79.99, 50, 'Informática'),
    ('Teclado Mecánico RGB', 'Teclado gaming con iluminación RGB', 129.99, 30, 'Informática'),
    ('Monitor Samsung 27"', 'Monitor Full HD IPS 27 pulgadas', 249.99, 20, 'Informática'),
    ('Auriculares Sony WH-1000XM4', 'Auriculares con cancelación de ruido', 299.99, 25, 'Audio'),
    ('Webcam Logitech C920', 'Webcam Full HD 1080p', 89.99, 40, 'Informática'),
    ('SSD Samsung 1TB', 'Disco duro sólido 1TB NVMe', 119.99, 35, 'Almacenamiento'),
    ('Cable HDMI 2m', 'Cable HDMI 2.0 alta velocidad', 12.99, 100, 'Accesorios')
]

# Insertamos todos los productos de una vez usando executemany()
# Es mucho más rápido que insertar uno por uno
cursor.executemany('''
    INSERT INTO Productos (nombre, descripcion, precio, stock, categoria)
    VALUES (?, ?, ?, ?, ?)
''', productos_datos)

# Confirmamos los cambios en la base de datos
conexion.commit()

print(f"✓ {len(productos_datos)} productos insertados correctamente")

## Paso 10: Consultar datos - SELECT básico

Ahora vamos a aprender a consultar los datos insertados.

In [ ]:
# Consultamos todos los clientes usando SELECT
# SELECT * significa "seleccionar todas las columnas"
# FROM Clientes especifica la tabla a consultar
cursor.execute('SELECT * FROM Clientes')

# fetchall() obtiene todos los resultados como una lista de tuplas
# Cada tupla representa un cliente con todos sus campos
clientes = cursor.fetchall()

print("CLIENTES REGISTRADOS:")
print("="*80)
# Iteramos sobre cada cliente
for cliente in clientes:
    # Accedemos a los campos por índice: [0]=id_cliente, [1]=nombre, [2]=email
    print(f"ID: {cliente[0]} | Nombre: {cliente[1]} | Email: {cliente[2]}")

# len() cuenta cuántos clientes hay en total
print(f"\nTotal de clientes: {len(clientes)}")

### Usar Pandas para mejor visualización

In [ ]:
# Usamos pandas para mostrar los resultados en formato tabla más legible
# read_sql_query() ejecuta la consulta SQL y devuelve un DataFrame de pandas
# Un DataFrame es una estructura de datos tabular (como una hoja de Excel)
# El segundo parámetro (conexion) indica qué base de datos usar
df_clientes = pd.read_sql_query('SELECT * FROM Clientes', conexion)

print("TABLA CLIENTES:")
# display() muestra el DataFrame en formato tabla bonito en Jupyter
display(df_clientes)

## Paso 11: Consultas con filtros - WHERE

In [ ]:
# Ejemplo 1: Buscar productos de una categoría específica
print("PRODUCTOS DE INFORMÁTICA:")
print("="*80)

# WHERE filtra los registros que cumplan una condición
# categoria = 'Informática' solo devuelve productos de esa categoría
# SELECT nombre, precio, stock: solo seleccionamos estas 3 columnas (no todas)
df_informatica = pd.read_sql_query('''
    SELECT nombre, precio, stock 
    FROM Productos 
    WHERE categoria = 'Informática'
''', conexion)

# Mostramos el resultado en formato tabla
display(df_informatica)

In [ ]:
# Ejemplo 2: Buscar productos con precio menor a 100€
print("PRODUCTOS ECONÓMICOS (< 100€):")
print("="*80)

# WHERE precio < 100: filtra productos cuyo precio sea menor a 100
# ORDER BY precio ASC: ordena los resultados por precio de menor a mayor
#   - ASC = ascendente (de menor a mayor)
#   - DESC = descendente (de mayor a menor)
df_economicos = pd.read_sql_query('''
    SELECT nombre, precio, categoria
    FROM Productos 
    WHERE precio < 100
    ORDER BY precio ASC
''', conexion)

display(df_economicos)

In [ ]:
# Ejemplo 3: Buscar productos con stock bajo (menos de 25 unidades)
print("PRODUCTOS CON STOCK BAJO:")
print("="*80)

# WHERE stock < 25: filtra productos con menos de 25 unidades en stock
# ORDER BY stock ASC: ordena del stock más bajo al más alto
# Esto es útil para saber qué productos necesitamos reponer urgentemente
df_stock_bajo = pd.read_sql_query('''
    SELECT nombre, stock, precio
    FROM Productos 
    WHERE stock < 25
    ORDER BY stock ASC
''', conexion)

display(df_stock_bajo)

## Paso 12: Crear pedidos (relaciones entre tablas)

Ahora vamos a crear pedidos relacionando clientes con productos.

In [ ]:
# PEDIDO 1: Juan Pérez compra una laptop y un mouse

# Primero insertamos el registro del pedido en la tabla Pedidos
# Este registro contiene la información general del pedido
# (1, ...) significa id_cliente=1 que corresponde a Juan Pérez
# 779.98 es el total del pedido (suma de los productos)
cursor.execute('''
    INSERT INTO Pedidos (id_cliente, fecha_pedido, estado, total)
    VALUES (?, ?, ?, ?)
''', (1, '2024-05-15', 'Completado', 779.98))

# Obtenemos el ID del pedido recién creado con lastrowid
# Necesitamos este ID para relacionar los productos con este pedido
id_pedido_1 = cursor.lastrowid

# Ahora insertamos los detalles del pedido en la tabla Detalles_Pedido
# Cada tupla representa un producto en el pedido
# Formato: (id_pedido, id_producto, cantidad, precio_unitario, subtotal)
detalles_pedido_1 = [
    (id_pedido_1, 1, 1, 699.99, 699.99),  # 1 Laptop HP (id_producto=1)
    (id_pedido_1, 2, 1, 79.99, 79.99)      # 1 Mouse Logitech (id_producto=2)
]

# Insertamos todos los detalles del pedido
cursor.executemany('''
    INSERT INTO Detalles_Pedido (id_pedido, id_producto, cantidad, precio_unitario, subtotal)
    VALUES (?, ?, ?, ?, ?)
''', detalles_pedido_1)

# Guardamos los cambios (tanto el pedido como sus detalles)
conexion.commit()
print(f"✓ Pedido #{id_pedido_1} creado para Juan Pérez")

In [ ]:
# PEDIDO 2: María García compra auriculares y un cable HDMI

# Insertamos el pedido principal para María García (id_cliente=2)
# El estado es 'En tránsito' indicando que el pedido ya fue enviado
cursor.execute('''
    INSERT INTO Pedidos (id_cliente, fecha_pedido, estado, total)
    VALUES (?, ?, ?, ?)
''', (2, '2024-05-20', 'En tránsito', 312.98))

# Guardamos el ID del nuevo pedido
id_pedido_2 = cursor.lastrowid

# Preparamos los detalles del pedido
# (id_pedido, id_producto, cantidad, precio_unitario, subtotal)
detalles_pedido_2 = [
    (id_pedido_2, 5, 1, 299.99, 299.99),  # 1 Auriculares Sony (id_producto=5)
    (id_pedido_2, 8, 1, 12.99, 12.99)      # 1 Cable HDMI (id_producto=8)
]

# Insertamos los detalles
cursor.executemany('''
    INSERT INTO Detalles_Pedido (id_pedido, id_producto, cantidad, precio_unitario, subtotal)
    VALUES (?, ?, ?, ?, ?)
''', detalles_pedido_2)

# Confirmamos los cambios en la base de datos
conexion.commit()
print(f"✓ Pedido #{id_pedido_2} creado para María García")

In [ ]:
# PEDIDO 3: Carlos Rodríguez compra un monitor, teclado y 2 cables HDMI

# Insertamos el pedido para Carlos Rodríguez (id_cliente=3)
# Estado 'Pendiente' significa que aún no se ha procesado o enviado
cursor.execute('''
    INSERT INTO Pedidos (id_cliente, fecha_pedido, estado, total)
    VALUES (?, ?, ?, ?)
''', (3, '2024-05-22', 'Pendiente', 405.96))

# Obtenemos el ID del pedido creado
id_pedido_3 = cursor.lastrowid

# Detalles del pedido
# Nota: En la última línea, cantidad=2 y subtotal=25.98 (2 cables × 12.99€)
detalles_pedido_3 = [
    (id_pedido_3, 4, 1, 249.99, 249.99),    # 1 Monitor Samsung (id_producto=4)
    (id_pedido_3, 3, 1, 129.99, 129.99),    # 1 Teclado Mecánico (id_producto=3)
    (id_pedido_3, 8, 2, 12.99, 25.98)       # 2 Cables HDMI (id_producto=8)
]

# Insertamos los detalles del pedido
cursor.executemany('''
    INSERT INTO Detalles_Pedido (id_pedido, id_producto, cantidad, precio_unitario, subtotal)
    VALUES (?, ?, ?, ?, ?)
''', detalles_pedido_3)

# Guardamos todos los cambios
conexion.commit()
print(f"✓ Pedido #{id_pedido_3} creado para Carlos Rodríguez")

## Paso 13: Consultas con JOIN (unir tablas)

Los JOIN nos permiten combinar información de varias tablas relacionadas.

### Consulta 1: Ver todos los pedidos con información del cliente

In [ ]:
# INNER JOIN combina las tablas Pedidos y Clientes usando id_cliente
# Esto nos permite ver información de ambas tablas en una sola consulta
# 
# FROM Pedidos p: seleccionamos de Pedidos (usamos alias 'p' para abreviar)
# INNER JOIN Clientes c: unimos con Clientes (alias 'c')
# ON p.id_cliente = c.id_cliente: la condición de unión (cómo se relacionan)
#
# c.nombre AS cliente: renombramos la columna 'nombre' a 'cliente' en el resultado
# ORDER BY p.fecha_pedido DESC: ordenamos por fecha, del más reciente al más antiguo
df_pedidos_completo = pd.read_sql_query('''
    SELECT 
        p.id_pedido,
        c.nombre AS cliente,
        c.email,
        p.fecha_pedido,
        p.estado,
        p.total
    FROM Pedidos p
    INNER JOIN Clientes c ON p.id_cliente = c.id_cliente
    ORDER BY p.fecha_pedido DESC
''', conexion)

print("TODOS LOS PEDIDOS CON INFORMACIÓN DEL CLIENTE:")
print("="*80)
display(df_pedidos_completo)

### Consulta 2: Detalle completo de un pedido específico

In [ ]:
# Consultamos el detalle del pedido #1 (Juan Pérez)
# Esta consulta une 3 tablas para mostrar información completa:
#   - Detalles_Pedido: contiene qué productos y cantidades
#   - Productos: contiene los nombres de los productos
#
# FROM Detalles_Pedido dp: tabla principal (alias 'dp')
# INNER JOIN Productos prod: unimos con Productos (alias 'prod')
# ON dp.id_producto = prod.id_producto: relacionamos por id_producto
# WHERE dp.id_pedido = 1: filtramos solo el pedido con ID 1
df_detalle_pedido = pd.read_sql_query('''
    SELECT 
        dp.id_pedido,
        prod.nombre AS producto,
        dp.cantidad,
        dp.precio_unitario,
        dp.subtotal
    FROM Detalles_Pedido dp
    INNER JOIN Productos prod ON dp.id_producto = prod.id_producto
    WHERE dp.id_pedido = 1
''', conexion)

print("DETALLE DEL PEDIDO #1 (Juan Pérez):")
print("="*80)
display(df_detalle_pedido)
# Calculamos el total del pedido sumando todos los subtotales
# .sum() suma todos los valores de la columna 'subtotal'
# :.2f formatea el número a 2 decimales
print(f"\nTOTAL DEL PEDIDO: {df_detalle_pedido['subtotal'].sum():.2f}€")

### Consulta 3: Resumen de compras por cliente

In [ ]:
# Usamos funciones de agregación para obtener resúmenes:
#   - COUNT(): cuenta el número de registros
#   - SUM(): suma valores numéricos
#   - AVG(): calcula el promedio
#   - MIN(): obtiene el valor mínimo
#   - MAX(): obtiene el valor máximo
#
# LEFT JOIN incluye TODOS los clientes, incluso los que no tienen pedidos
# (a diferencia de INNER JOIN que solo incluye clientes con pedidos)
#
# GROUP BY agrupa los resultados por cliente
# Esto permite calcular el total de pedidos y gasto por cada cliente
# ORDER BY total_gastado DESC: ordena de mayor a menor gasto
df_resumen_clientes = pd.read_sql_query('''
    SELECT 
        c.nombre AS cliente,
        COUNT(p.id_pedido) AS total_pedidos,
        SUM(p.total) AS total_gastado
    FROM Clientes c
    LEFT JOIN Pedidos p ON c.id_cliente = p.id_cliente
    GROUP BY c.id_cliente, c.nombre
    ORDER BY total_gastado DESC
''', conexion)

print("RESUMEN DE COMPRAS POR CLIENTE:")
print("="*80)
display(df_resumen_clientes)

### Consulta 4: Productos más vendidos

In [ ]:
# Consultamos qué productos se han vendido más
# Esta consulta nos ayuda a identificar los productos más populares
#
# SUM(dp.cantidad): suma todas las cantidades vendidas de cada producto
# SUM(dp.subtotal): suma los ingresos generados por cada producto
# GROUP BY agrupa por producto para calcular totales de cada uno
# ORDER BY unidades_vendidas DESC: ordena del más vendido al menos vendido
df_productos_vendidos = pd.read_sql_query('''
    SELECT 
        prod.nombre AS producto,
        SUM(dp.cantidad) AS unidades_vendidas,
        SUM(dp.subtotal) AS ingresos_totales
    FROM Detalles_Pedido dp
    INNER JOIN Productos prod ON dp.id_producto = prod.id_producto
    GROUP BY prod.id_producto, prod.nombre
    ORDER BY unidades_vendidas DESC
''', conexion)

print("PRODUCTOS MÁS VENDIDOS:")
print("="*80)
display(df_productos_vendidos)

## Paso 14: Actualizar datos - UPDATE

In [ ]:
# Ejemplo 1: Cambiar el estado de un pedido
# Primero consultamos el estado actual del pedido #3
print("Estado del pedido #3 antes:")
cursor.execute('SELECT estado FROM Pedidos WHERE id_pedido = 3')
# fetchone() obtiene solo el primer resultado (una tupla)
# [0] accede al primer elemento de la tupla (el estado)
print(f"  Estado: {cursor.fetchone()[0]}")

# UPDATE modifica registros existentes en una tabla
# SET especifica qué campo(s) queremos cambiar y su nuevo valor
# WHERE especifica qué registros modificar (sin WHERE se modificarían TODOS)
cursor.execute('''
    UPDATE Pedidos 
    SET estado = 'En tránsito'
    WHERE id_pedido = 3
''')

# Guardamos el cambio
conexion.commit()

# Verificamos que el cambio se aplicó correctamente
print("\nEstado del pedido #3 después:")
cursor.execute('SELECT estado FROM Pedidos WHERE id_pedido = 3')
print(f"  Estado: {cursor.fetchone()[0]}")
print("\n✓ Estado actualizado correctamente")

In [ ]:
# Ejemplo 2: Actualizar el stock después de una venta
# Consultamos el stock actual del Monitor Samsung (id_producto=4)
print("Stock del Monitor Samsung antes:")
cursor.execute('SELECT stock FROM Productos WHERE id_producto = 4')
print(f"  Stock: {cursor.fetchone()[0]} unidades")

# Reducimos el stock en 1 unidad porque se vendió 1 monitor
# SET stock = stock - 1: toma el valor actual y le resta 1
# Esto es mejor que poner un número fijo porque no necesitamos saber el stock actual
cursor.execute('''
    UPDATE Productos 
    SET stock = stock - 1
    WHERE id_producto = 4
''')

# Confirmamos el cambio
conexion.commit()

# Verificamos el nuevo stock
print("\nStock del Monitor Samsung después:")
cursor.execute('SELECT stock FROM Productos WHERE id_producto = 4')
print(f"  Stock: {cursor.fetchone()[0]} unidades")
print("\n✓ Stock actualizado correctamente")

In [ ]:
# Ejemplo 3: Actualizar el precio de un producto
# Consultamos el precio actual del Cable HDMI (id_producto=8)
print("Precio del Cable HDMI antes:")
cursor.execute('SELECT precio FROM Productos WHERE id_producto = 8')
print(f"  Precio: {cursor.fetchone()[0]}€")

# Aplicamos un descuento del 20%
# precio * 0.8 equivale a reducir el precio al 80% del original
# (0.8 = 1 - 0.2, donde 0.2 es el 20% de descuento)
cursor.execute('''
    UPDATE Productos 
    SET precio = precio * 0.8
    WHERE id_producto = 8
''')

# Guardamos el cambio
conexion.commit()

# Verificamos el nuevo precio
print("\nPrecio del Cable HDMI después (20% descuento):")
cursor.execute('SELECT precio FROM Productos WHERE id_producto = 8')
# :.2f formatea el número a 2 decimales (ej: 10.392 → 10.39)
print(f"  Precio: {cursor.fetchone()[0]:.2f}€")
print("\n✓ Precio actualizado correctamente")

## Paso 15: Eliminar datos - DELETE

In [ ]:
# Primero insertamos un cliente temporal para luego eliminarlo
# Este es solo un ejemplo didáctico para mostrar cómo funciona DELETE
cursor.execute('''
    INSERT INTO Clientes (nombre, email, telefono, direccion, fecha_registro)
    VALUES (?, ?, ?, ?, ?)
''', ('Cliente Temporal', 'temporal@email.com', '600000000', 'Dirección Temporal', '2024-05-25'))

# Guardamos el ID del cliente temporal
id_temporal = cursor.lastrowid
conexion.commit()

print(f"✓ Cliente temporal creado con ID: {id_temporal}")

# DELETE elimina registros de una tabla
# WHERE especifica qué registros eliminar
# IMPORTANTE: Sin WHERE se eliminarían TODOS los registros de la tabla
# (?,) es una tupla con un solo elemento (nota la coma)
cursor.execute('DELETE FROM Clientes WHERE id_cliente = ?', (id_temporal,))
conexion.commit()

print(f"✓ Cliente temporal eliminado")

# Verificamos que se eliminó correctamente
# COUNT(*) cuenta cuántos registros cumplen la condición
cursor.execute('SELECT COUNT(*) FROM Clientes WHERE id_cliente = ?', (id_temporal,))
count = cursor.fetchone()[0]
print(f"\nVerificación: {count} clientes encontrados con ese ID (debería ser 0)")

## Paso 16: Consultas avanzadas - Subconsultas

In [ ]:
# Ejemplo: Encontrar clientes que han gastado más que el promedio
# Este es un ejemplo de SUBCONSULTA (query dentro de otra query)

# Primero calculamos el gasto promedio de todos los pedidos
# AVG() calcula el promedio (average)
cursor.execute('''
    SELECT AVG(total) FROM Pedidos
''')
promedio = cursor.fetchone()[0]
print(f"Gasto promedio por pedido: {promedio:.2f}€\n")

# Ahora buscamos pedidos con total superior al promedio
# WHERE p.total > (SELECT AVG(total) FROM Pedidos): subconsulta
# La subconsulta se ejecuta primero y devuelve un valor (el promedio)
# Luego se compara cada pedido con ese valor
# Esto es útil para identificar clientes premium o pedidos importantes
df_clientes_premium = pd.read_sql_query('''
    SELECT 
        c.nombre AS cliente,
        p.id_pedido,
        p.total
    FROM Clientes c
    INNER JOIN Pedidos p ON c.id_cliente = p.id_cliente
    WHERE p.total > (SELECT AVG(total) FROM Pedidos)
    ORDER BY p.total DESC
''', conexion)

print("CLIENTES CON PEDIDOS SUPERIORES AL PROMEDIO:")
print("="*80)
display(df_clientes_premium)

## Paso 17: Funciones de agregación y estadísticas

In [ ]:
# Obtenemos estadísticas generales de la tienda usando funciones de agregación
# Todas estas funciones se calculan en una sola consulta para eficiencia
#
# Funciones de agregación:
#   - COUNT(*): cuenta el número total de pedidos
#   - SUM(total): suma todos los totales (ingresos totales)
#   - AVG(total): calcula el promedio de los totales (ticket promedio)
#   - MIN(total): encuentra el pedido de menor valor
#   - MAX(total): encuentra el pedido de mayor valor
cursor.execute('''
    SELECT 
        COUNT(*) AS total_pedidos,
        SUM(total) AS ingresos_totales,
        AVG(total) AS ticket_promedio,
        MIN(total) AS pedido_minimo,
        MAX(total) AS pedido_maximo
    FROM Pedidos
''')

# Obtenemos la fila con todas las estadísticas
stats = cursor.fetchone()

print("ESTADÍSTICAS DE LA TIENDA:")
print("="*80)
# Accedemos a cada estadística por su índice
print(f"Total de pedidos: {stats[0]}")
print(f"Ingresos totales: {stats[1]:.2f}€")
print(f"Ticket promedio: {stats[2]:.2f}€")
print(f"Pedido mínimo: {stats[3]:.2f}€")
print(f"Pedido máximo: {stats[4]:.2f}€")

In [ ]:
# Estadísticas por categoría de productos
# Esta consulta agrupa productos por categoría y calcula métricas para cada una
#
# COUNT(*): cuenta cuántos productos hay en cada categoría
# AVG(precio): calcula el precio promedio de los productos en cada categoría
# SUM(stock): suma el stock total de cada categoría
# GROUP BY categoria: agrupa los resultados por categoría
# Esto nos permite ver un resumen de cada categoría en lugar de productos individuales
df_stats_categoria = pd.read_sql_query('''
    SELECT 
        categoria,
        COUNT(*) AS cantidad_productos,
        AVG(precio) AS precio_promedio,
        SUM(stock) AS stock_total
    FROM Productos
    GROUP BY categoria
    ORDER BY cantidad_productos DESC
''', conexion)

print("\nESTADÍSTICAS POR CATEGORÍA:")
print("="*80)
display(df_stats_categoria)

## Paso 18: Crear una función auxiliar para consultas comunes

Vamos a crear funciones Python que encapsulen operaciones comunes de la base de datos.

In [ ]:
def buscar_cliente_por_email(email):
    """
    Busca un cliente por su email
    
    Esta función encapsula una consulta SQL común en una función Python reutilizable.
    Es una buena práctica para evitar repetir código.
    
    Args:
        email (str): Email del cliente a buscar
    
    Returns:
        tuple: Tupla con los datos del cliente (id, nombre, email, etc.) 
               o None si no existe ningún cliente con ese email
    """
    # Ejecutamos la consulta usando el parámetro email
    cursor.execute('SELECT * FROM Clientes WHERE email = ?', (email,))
    # fetchone() devuelve None si no hay resultados
    return cursor.fetchone()

def obtener_pedidos_cliente(id_cliente):
    """
    Obtiene todos los pedidos de un cliente específico
    
    Args:
        id_cliente (int): ID del cliente
    
    Returns:
        DataFrame: DataFrame de pandas con los pedidos del cliente
                   ordenados por fecha (más reciente primero)
    """
    query = '''
        SELECT 
            id_pedido,
            fecha_pedido,
            estado,
            total
        FROM Pedidos
        WHERE id_cliente = ?
        ORDER BY fecha_pedido DESC
    '''
    # params=(id_cliente,) pasa el parámetro a la consulta
    return pd.read_sql_query(query, conexion, params=(id_cliente,))

def crear_pedido(id_cliente, productos_lista):
    """
    Crea un nuevo pedido con múltiples productos de forma automática
    
    Esta función simplifica la creación de pedidos al:
    1. Calcular automáticamente el total
    2. Crear el registro del pedido
    3. Crear todos los detalles del pedido
    
    Args:
        id_cliente (int): ID del cliente que realiza el pedido
        productos_lista (list): Lista de tuplas (id_producto, cantidad)
                               Ejemplo: [(1, 2), (3, 1)] = 2 unidades del producto 1 y 1 del producto 3
    
    Returns:
        int: ID del pedido creado
    """
    # Calculamos el total del pedido sumando todos los subtotales
    total = 0
    detalles = []
    
    # Iteramos sobre cada producto en la lista
    for id_producto, cantidad in productos_lista:
        # Obtenemos el precio actual del producto desde la base de datos
        cursor.execute('SELECT precio FROM Productos WHERE id_producto = ?', (id_producto,))
        precio = cursor.fetchone()[0]
        # Calculamos el subtotal (precio × cantidad)
        subtotal = precio * cantidad
        total += subtotal
        # Guardamos los detalles para insertarlos después
        detalles.append((id_producto, cantidad, precio, subtotal))
    
    # Insertamos el pedido principal
    # datetime('now') obtiene la fecha y hora actual automáticamente
    cursor.execute('''
        INSERT INTO Pedidos (id_cliente, fecha_pedido, estado, total)
        VALUES (?, datetime('now'), 'Pendiente', ?)
    ''', (id_cliente, total))
    
    # Obtenemos el ID del pedido recién creado
    id_pedido = cursor.lastrowid
    
    # Insertamos todos los detalles del pedido
    for id_producto, cantidad, precio, subtotal in detalles:
        cursor.execute('''
            INSERT INTO Detalles_Pedido (id_pedido, id_producto, cantidad, precio_unitario, subtotal)
            VALUES (?, ?, ?, ?, ?)
        ''', (id_pedido, id_producto, cantidad, precio, subtotal))
    
    # Guardamos todos los cambios (pedido + detalles)
    conexion.commit()
    return id_pedido

print("✓ Funciones auxiliares creadas correctamente")

### Probar las funciones auxiliares

In [ ]:
# Ejemplo 1: Buscar un cliente por email usando nuestra función
# Llamamos a la función que creamos anteriormente
cliente = buscar_cliente_por_email('juan.perez@email.com')

# Mostramos los datos del cliente encontrado
print("CLIENTE ENCONTRADO:")
# Accedemos a cada campo por su índice en la tupla
print(f"  ID: {cliente[0]}")
print(f"  Nombre: {cliente[1]}")
print(f"  Email: {cliente[2]}")
print(f"  Teléfono: {cliente[3]}")

In [ ]:
# Ejemplo 2: Obtener todos los pedidos de un cliente
# ID 1 corresponde a Juan Pérez
pedidos_juan = obtener_pedidos_cliente(1)

print("PEDIDOS DE JUAN PÉREZ:")
print("="*80)
# La función nos devuelve un DataFrame listo para mostrar
display(pedidos_juan)

In [ ]:
# Ejemplo 3: Crear un nuevo pedido usando nuestra función auxiliar
# Ana Martínez (ID=4) compra una webcam y un SSD
# La función crea_pedido se encarga de:
#   1. Buscar los precios actuales de los productos
#   2. Calcular los subtotales y el total
#   3. Crear el pedido y sus detalles
# Todo esto con una sola llamada, mucho más simple que hacerlo manualmente
nuevo_pedido = crear_pedido(
    id_cliente=4,
    productos_lista=[
        (6, 1),  # (id_producto, cantidad): 1 Webcam Logitech (id_producto=6)
        (7, 1)   # 1 SSD Samsung (id_producto=7)
    ]
)

print(f"✓ Nuevo pedido #{nuevo_pedido} creado para Ana Martínez")

# Consultamos y mostramos el detalle del pedido recién creado
# Usamos f-string para incluir el ID del pedido en la consulta
df_nuevo = pd.read_sql_query(f'''
    SELECT 
        dp.id_pedido,
        prod.nombre AS producto,
        dp.cantidad,
        dp.precio_unitario,
        dp.subtotal
    FROM Detalles_Pedido dp
    INNER JOIN Productos prod ON dp.id_producto = prod.id_producto
    WHERE dp.id_pedido = {nuevo_pedido}
''', conexion)

print("\nDETALLE DEL PEDIDO CREADO:")
display(df_nuevo)

## Paso 19: Transacciones (garantizar consistencia de datos)

Las transacciones permiten agrupar múltiples operaciones y revertirlas si algo falla.

In [ ]:
# Ejemplo de TRANSACCIÓN: transferir stock entre productos
# Una transacción agrupa múltiples operaciones en una sola unidad atómica:
#   - O se ejecutan TODAS las operaciones
#   - O NO se ejecuta NINGUNA (si hay error)
# Esto garantiza la consistencia de los datos

try:
    # BEGIN TRANSACTION inicia una transacción explícita
    # Todas las operaciones posteriores son tentativas hasta el commit()
    cursor.execute('BEGIN TRANSACTION')
    
    print("Iniciando transferencia de stock...\n")
    
    # Consultamos el stock actual de los productos involucrados
    cursor.execute('SELECT nombre, stock FROM Productos WHERE id_producto IN (2, 3)')
    print("STOCK ANTES:")
    for prod in cursor.fetchall():
        print(f"  {prod[0]}: {prod[1]} unidades")
    
    # Reducimos stock del Mouse (ID=2) en 5 unidades
    cursor.execute('UPDATE Productos SET stock = stock - 5 WHERE id_producto = 2')
    
    # Aumentamos stock del Teclado (ID=3) en 5 unidades
    # Es como si transfiriéramos stock de un producto a otro
    cursor.execute('UPDATE Productos SET stock = stock + 5 WHERE id_producto = 3')
    
    # Si llegamos aquí sin errores, confirmamos la transacción
    # commit() hace permanentes todos los cambios
    conexion.commit()
    
    # Consultamos el stock después de la transferencia
    cursor.execute('SELECT nombre, stock FROM Productos WHERE id_producto IN (2, 3)')
    print("\nSTOCK DESPUÉS:")
    for prod in cursor.fetchall():
        print(f"  {prod[0]}: {prod[1]} unidades")
    
    print("\n✓ Transferencia completada exitosamente")
    
except Exception as e:
    # Si ocurre CUALQUIER error, revertimos TODOS los cambios
    # rollback() deshace todas las operaciones de la transacción
    # La base de datos vuelve al estado anterior al BEGIN TRANSACTION
    conexion.rollback()
    print(f"✗ Error en la transacción: {e}")
    print("Cambios revertidos")

## Paso 20: Exportar datos a CSV

In [ ]:
# Exportamos datos de la base de datos a archivos CSV
# CSV (Comma-Separated Values) es un formato universal que puede abrirse en Excel

# Exportamos la tabla de productos completa a un archivo CSV
df_productos = pd.read_sql_query('SELECT * FROM Productos', conexion)
# to_csv() guarda el DataFrame en un archivo CSV
# index=False: no incluye el índice de pandas en el archivo
# encoding='utf-8': asegura que los caracteres especiales (ñ, acentos) se guarden correctamente
df_productos.to_csv('productos_export.csv', index=False, encoding='utf-8')
print("✓ Productos exportados a 'productos_export.csv'")

# Exportamos un reporte personalizado de pedidos
# Este CSV combina información de pedidos y clientes
df_reporte = pd.read_sql_query('''
    SELECT 
        p.id_pedido,
        c.nombre AS cliente,
        p.fecha_pedido,
        p.estado,
        p.total
    FROM Pedidos p
    INNER JOIN Clientes c ON p.id_cliente = c.id_cliente
''', conexion)
df_reporte.to_csv('reporte_pedidos.csv', index=False, encoding='utf-8')
print("✓ Reporte de pedidos exportado a 'reporte_pedidos.csv'")

## Paso 21: Crear índices para mejorar rendimiento

Los índices aceleran las consultas sobre columnas que se buscan frecuentemente.

In [ ]:
# Creamos ÍNDICES para mejorar el rendimiento de las consultas
# Un índice es como el índice de un libro: permite encontrar información más rápido
# Sin índice, la base de datos debe revisar TODOS los registros (búsqueda secuencial)
# Con índice, puede ir directamente a los registros relevantes (búsqueda optimizada)

# Índice en el campo email de la tabla Clientes
# IF NOT EXISTS evita error si el índice ya existe
# Útil porque frecuentemente buscamos clientes por email
cursor.execute('CREATE INDEX IF NOT EXISTS idx_clientes_email ON Clientes(email)')
print("✓ Índice creado en Clientes.email")

# Índice en el campo categoría de Productos
# Acelera búsquedas como "todos los productos de Informática"
cursor.execute('CREATE INDEX IF NOT EXISTS idx_productos_categoria ON Productos(categoria)')
print("✓ Índice creado en Productos.categoria")

# Índice en el estado de Pedidos
# Útil para consultas como "todos los pedidos pendientes"
cursor.execute('CREATE INDEX IF NOT EXISTS idx_pedidos_estado ON Pedidos(estado)')
print("✓ Índice creado en Pedidos.estado")

# Índice COMPUESTO (en múltiples columnas)
# Optimiza consultas que filtran por cliente Y fecha simultáneamente
# Ejemplo: "todos los pedidos del cliente 5 en marzo"
cursor.execute('CREATE INDEX IF NOT EXISTS idx_pedidos_cliente_fecha ON Pedidos(id_cliente, fecha_pedido)')
print("✓ Índice compuesto creado en Pedidos(id_cliente, fecha_pedido)")

# Guardamos la creación de los índices
conexion.commit()
print("\n✓ Todos los índices creados correctamente")

## Paso 22: Consulta de metadatos de la base de datos

In [ ]:
# Obtener información sobre la estructura de todas las tablas
# sqlite_master es una tabla especial del sistema que contiene metadatos
# Contiene información sobre todas las tablas, índices, vistas, etc.
# type='table': filtramos solo las tablas (no índices ni otros objetos)
# sql: columna que contiene el comando CREATE TABLE original
cursor.execute("SELECT name, sql FROM sqlite_master WHERE type='table'")
tablas_info = cursor.fetchall()

print("ESTRUCTURA DE LA BASE DE DATOS:")
print("="*80)
# Iteramos sobre cada tabla y mostramos su definición SQL completa
for nombre, sql in tablas_info:
    print(f"\nTabla: {nombre}")
    print("-" * 80)
    # sql contiene el comando CREATE TABLE completo que creó la tabla
    print(sql)

In [ ]:
# Obtener información sobre todos los índices en la base de datos
# type='index': filtramos solo los índices
# Los índices automáticos (creados por PRIMARY KEY) no tienen SQL
cursor.execute("SELECT name, sql FROM sqlite_master WHERE type='index'")
indices_info = cursor.fetchall()

print("\nÍNDICES EN LA BASE DE DATOS:")
print("="*80)
for nombre, sql in indices_info:
    # if sql: filtra índices automáticos que tienen sql=None
    if sql:
        print(f"\n{nombre}:")
        # Mostramos el comando CREATE INDEX que creó el índice
        print(f"  {sql}")

## Paso 23: Resumen final y estadísticas completas

In [ ]:
print("="*80)
print("RESUMEN COMPLETO DE LA BASE DE DATOS")
print("="*80)

# Contamos registros en cada tabla usando COUNT(*)
# COUNT(*) devuelve el número total de filas en la tabla
cursor.execute('SELECT COUNT(*) FROM Clientes')
print(f"\nTotal de clientes: {cursor.fetchone()[0]}")

cursor.execute('SELECT COUNT(*) FROM Productos')
print(f"Total de productos: {cursor.fetchone()[0]}")

cursor.execute('SELECT COUNT(*) FROM Pedidos')
print(f"Total de pedidos: {cursor.fetchone()[0]}")

cursor.execute('SELECT COUNT(*) FROM Detalles_Pedido')
print(f"Total de líneas de pedido: {cursor.fetchone()[0]}")

# Estadísticas financieras de la tienda
# SUM(total): suma todos los totales de pedidos = ingresos totales
cursor.execute('SELECT SUM(total) FROM Pedidos')
print(f"\nIngresos totales: {cursor.fetchone()[0]:.2f}€")

# AVG(total): calcula el ticket promedio (gasto promedio por pedido)
cursor.execute('SELECT AVG(total) FROM Pedidos')
print(f"Ticket promedio: {cursor.fetchone()[0]:.2f}€")

# Calculamos el valor total del inventario
# precio * stock da el valor de cada producto
# SUM() suma el valor de todos los productos
cursor.execute('SELECT SUM(precio * stock) FROM Productos')
print(f"Valor del inventario: {cursor.fetchone()[0]:.2f}€")

print("\n" + "="*80)

## Paso 24: Cerrar la conexión

**IMPORTANTE**: Siempre debemos cerrar la conexión cuando terminamos de trabajar con la base de datos.

In [ ]:
# IMPORTANTE: Siempre cerrar la conexión cuando terminamos
# Esto libera recursos del sistema y asegura que todos los cambios se guarden

# Cerramos el cursor primero
# El cursor ya no podrá ejecutar más comandos SQL después de esto
cursor.close()

# Cerramos la conexión a la base de datos
# Esto guarda cualquier cambio pendiente y libera el archivo de la base de datos
conexion.close()

print("✓ Conexión cerrada correctamente")
print("\nLa base de datos 'tienda_online.db' ha sido guardada en el disco.")
print("Puedes abrir este archivo con cualquier cliente de SQLite para explorar los datos.")

## Resumen de conceptos aprendidos

### Comandos SQL principales:
1. **CREATE TABLE**: Crear tablas con campos y restricciones
2. **INSERT INTO**: Insertar datos en las tablas
3. **SELECT**: Consultar datos (con WHERE, ORDER BY, GROUP BY)
4. **UPDATE**: Actualizar registros existentes
5. **DELETE**: Eliminar registros
6. **JOIN**: Unir tablas relacionadas (INNER JOIN, LEFT JOIN)
7. **INDEX**: Crear índices para mejorar rendimiento

### Conceptos de bases de datos relacionales:
- **Clave primaria (PRIMARY KEY)**: Identificador único de cada registro
- **Clave foránea (FOREIGN KEY)**: Relación entre tablas
- **Restricciones**: NOT NULL, UNIQUE, DEFAULT
- **Relaciones**: Uno a muchos, muchos a muchos
- **Integridad referencial**: ON DELETE CASCADE

### Operaciones con Python:
- Conectar a la base de datos con `sqlite3.connect()`
- Ejecutar comandos SQL con `cursor.execute()`
- Obtener resultados con `fetchone()`, `fetchall()`
- Confirmar cambios con `commit()`
- Revertir cambios con `rollback()`
- Usar pandas para mejor visualización
- Crear funciones auxiliares para operaciones comunes

### Buenas prácticas:
- Siempre usar parámetros (`?`) en lugar de concatenar strings (evita inyección SQL)
- Cerrar conexiones cuando ya no se necesiten
- Usar transacciones para operaciones críticas
- Crear índices en columnas frecuentemente consultadas
- Normalizar las tablas para evitar redundancia

---

**¡Felicidades!** Has completado el tutorial de bases de datos SQL relacionales con Python.

Ahora tienes los conocimientos básicos para crear, gestionar y consultar bases de datos en tus propios proyectos.